In [1]:
from pathlib import Path
import os
os.getcwd()

'/home/sakamoto/spectrum_re/note/data_prepare'

In [3]:
import sys

module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, module_path)

import note.base as base
sys.path.pop(0)

SEED = 42
base.fix_seed(SEED)

2024-07-30 11:07:08.360041: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 11:07:08.380380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 11:07:08.402280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 11:07:08.408436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 11:07:08.427693: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import zipfile

zip_path = Path('../../data/RLdata/RIKEN_LIPIDOMICS_dataset.zip')
extract_to = Path('../../data/RLdata/RIKEN_LIPIDOMICS_dataset')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [4]:
extract_to = Path('../../data/RLdata/RIKEN_LIPIDOMICS_dataset')
for path in list(extract_to.rglob('*.xlsx')):
    dataframe = base.loadDataframe_for_RL(str(path))
    base.insertdata(dataframe, path='../../data/RLdata/ms2_rikenlipid.db')

In [5]:
def get_fileid(path):
    path_elements = path.split("/")
    file_name = path_elements[-1]
    file_name_elements = file_name.split("_")
    return file_name_elements[0]

In [14]:
import pandas as pd

df = base.import_sqlite3_data('../../data/RLdata/ms2_rikenlipid.db').drop_duplicates().reset_index(drop=True)
df['dataset'] = df['dataset'].str.replace('../../data/RLdata/RIKEN_LIPIDOMICS_dataset/dataset/', '', regex=False)
df['id'] = df['dataset'].apply(get_fileid).astype('int')
df = df[~df['Ontology'].str.contains('IS \(')]
df['Comment'] = df['Comment'].fillna('NC')
df = df[~(df['Comment'].str.contains('IS \(') | df['Comment'].str.contains('d') | df['Comment'].str.contains('two peaks'))].reset_index(drop=True)

In [18]:
df_pos = df[df['Adducttype'].str.contains('\]\+')  |  df['Adducttype'].str.contains('\]2\+')]
df_neg = df[df['Adducttype'].str.contains('\]\-') |  df['Adducttype'].str.contains('\]2\-')]
print(len(df), len(df_pos)+len(df_neg), len(df_neg), len(df_pos))

79747 79747 40790 38957


In [41]:
df_neg2 = df_neg[['Metabolitename', 'Ontology', 'id', 'AlignmentID','AverageMz', 'MSMSspectrum']].rename(columns={'id':'dataset'}).dropna().drop_duplicates().reset_index(drop=True)
df_pos2 = df_pos[['Metabolitename', 'Ontology', 'id', 'AlignmentID','AverageMz', 'MSMSspectrum']].rename(columns={'id':'dataset'}).dropna().drop_duplicates().reset_index(drop=True)

In [43]:
df_neg_exp_ = base.spectrum_tidy(df_neg2)
df_neg_neuloss_ = base.neutralloss_tidy(df_neg2)
df_neg_data = base.spectrum_neutralloss_table(df_neg_exp_, df_neg_neuloss_)    
df_neg_data['Ontology'] = df_neg_data['Ontology'].str.replace(r'-', '_')

In [46]:
df_pos_exp_ = base.spectrum_tidy(df_pos2)
df_pos_neuloss_ = base.neutralloss_tidy(df_pos2)
df_pos_data = base.spectrum_neutralloss_table(df_pos_exp_, df_pos_neuloss_)    
df_pos_data['Ontology'] = df_pos_data['Ontology'].str.replace(r'-', '_')

In [47]:
df_pos_data.to_csv('../../data/RLdata/df_RLtable_pos.csv', index=False) 
df_neg_data.to_csv('../../data/RLdata/df_RLtable_neg.csv', index=False)